<a href="https://colab.research.google.com/github/abnerxch/bi-project/blob/main/ETL_de_SARS_CoV_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adriana Mundo, Ábner Xocop y Fernanda González
# Epidemiología social del SARS-CoV-2: ETL de casos

## Prep env

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta 
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Extract

In [ ]:
# del 01/22/2020 a ayer
dia_inicial = datetime(2020,1,22,0,0,0)
yesterday = datetime.now() - timedelta(days=1)
#dias_transcurridos = (datetime.now() - dia_inicial).days 
dias_transcurridos = (yesterday - dia_inicial).days 

df_raw = pd.DataFrame()
date = dia_inicial
urls = []

In [ ]:
for i in range(0, dias_transcurridos):
    url_str = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
    if date.month < 10:
        url_str = url_str + '0' + str(date.month) + '-'
    else:
        url_str = url_str + str(date.month) + '-'
    if date.day < 10:
        url_str = url_str + '0' + str(date.day) + '-'
    else:
        url_str = url_str + str(date.day) + '-'
    url_str = url_str + str(date.year) + '.csv'
    urls.append(url_str)
    df_dia = pd.read_csv(url_str).fillna(0)
    df_raw = df_raw.append(df_dia, ignore_index = True)    
    date += timedelta(days=1)
    #print(date)

df_raw.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio
0,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_raw.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/CovidRaw.csv', index = False)

In [ ]:
# alternative extract, faster!
# df_raw = pd.read_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/CovidRaw.csv')

## Transform

### Timeline of cases per State

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609667 entries, 0 to 1609666
Data columns (total 21 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Province/State       7677 non-null     object 
 1   Country/Region       7677 non-null     object 
 2   Last Update          7677 non-null     object 
 3   Confirmed            1609667 non-null  float64
 4   Deaths               1609667 non-null  float64
 5   Recovered            1609667 non-null  float64
 6   Latitude             4820 non-null     float64
 7   Longitude            4820 non-null     float64
 8   FIPS                 1601990 non-null  float64
 9   Admin2               1601990 non-null  object 
 10  Province_State       1601990 non-null  object 
 11  Country_Region       1601990 non-null  object 
 12  Last_Update          1601990 non-null  object 
 13  Lat                  1601990 non-null  float64
 14  Long_                1601990 non-null  float64
 15

In [ ]:
# filter US rows
df_us = df_raw[(df_raw['Country_Region'] == 'US') | (df_raw['Country/Region'] == 'US')]

# combine columns & drop useless
df = pd.DataFrame()
df['LastUpdate'] = df_us['Last Update'].fillna(df_us['Last_Update'])
df['LastUpdate'] = pd.to_datetime(df['LastUpdate']).dt.date
df['StateName'] = df_us['Province/State'].fillna(df_us['Province_State'])
df['Confirmed'] = df_us['Confirmed']
df['Deaths'] = df_us['Deaths']
df['IncidenceRate'] = df_us['Incident_Rate'].fillna(df_us['Incidence_Rate'])
df['CaseFatalityRatio'] = df_us['Case-Fatality_Ratio'].fillna(df_us['Case_Fatality_Ratio'])
df.info()
df.head(-10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1340356 entries, 31 to 1609612
Data columns (total 6 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   LastUpdate         1340356 non-null  object 
 1   StateName          1340356 non-null  object 
 2   Confirmed          1340356 non-null  float64
 3   Deaths             1340356 non-null  float64
 4   IncidenceRate      1143216 non-null  float64
 5   CaseFatalityRatio  1143216 non-null  object 
dtypes: float64(3), object(3)
memory usage: 71.6+ MB


,LastUpdate,StateName,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio
31,2020-01-22,Washington,1.0,0.0,NaN,NaN
70,2020-01-23,Washington,1.0,0.0,NaN,NaN
119,2020-01-24,Washington,1.0,0.0,NaN,NaN
120,2020-01-24,Chicago,1.0,0.0,NaN,NaN
161,2020-01-25,Washington,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...
1609598,2021-05-16,Wyoming,718.0,10.0,8502.072232,1.39276
1609599,2021-05-16,Wyoming,9212.0,114.0,9258.291457,1.23752
1609600,2021-05-16,Wyoming,1412.0,12.0,7120.524458,0.849858
1609601,2021-05-16,Wyoming,8039.0,136.0,10066.618247,1.69175


In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'United States Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

df['State'] = df['StateName'].map(us_state_abbrev)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1340356 entries, 31 to 1609612
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   LastUpdate         1340356 non-null  object 
 1   StateName          1340356 non-null  object 
 2   Confirmed          1340356 non-null  float64
 3   Deaths             1340356 non-null  float64
 4   IncidenceRate      1143216 non-null  float64
 5   CaseFatalityRatio  1143216 non-null  object 
 6   State              1338197 non-null  object 
dtypes: float64(3), object(4)
memory usage: 81.8+ MB


,LastUpdate,StateName,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio,State
31,2020-01-22,Washington,1.0,0.0,NaN,NaN,WA
70,2020-01-23,Washington,1.0,0.0,NaN,NaN,WA
119,2020-01-24,Washington,1.0,0.0,NaN,NaN,WA
120,2020-01-24,Chicago,1.0,0.0,NaN,NaN,NaN
161,2020-01-25,Washington,1.0,0.0,NaN,NaN,WA


In [ ]:
df = df[df['State'].notna()]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338197 entries, 31 to 1609612
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   LastUpdate         1338197 non-null  object 
 1   StateName          1338197 non-null  object 
 2   Confirmed          1338197 non-null  float64
 3   Deaths             1338197 non-null  float64
 4   IncidenceRate      1142160 non-null  float64
 5   CaseFatalityRatio  1142160 non-null  object 
 6   State              1338197 non-null  object 
dtypes: float64(3), object(4)
memory usage: 81.7+ MB


,LastUpdate,StateName,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio,State
31,2020-01-22,Washington,1.0,0.0,NaN,NaN,WA
70,2020-01-23,Washington,1.0,0.0,NaN,NaN,WA
119,2020-01-24,Washington,1.0,0.0,NaN,NaN,WA
161,2020-01-25,Washington,1.0,0.0,NaN,NaN,WA
162,2020-01-25,Illinois,1.0,0.0,NaN,NaN,IL


In [ ]:
df = df[['LastUpdate', 'State', 'Confirmed', 'Deaths', 'IncidenceRate', 'CaseFatalityRatio']]
df['CaseFatalityRatio'] = pd.to_numeric(df['CaseFatalityRatio'])
df_timeline = df.groupby(['LastUpdate', 'State'], as_index=False).agg(
    {
         'Confirmed': 'sum',
         'Deaths': 'sum',
         'IncidenceRate': 'mean',
         'CaseFatalityRatio': 'mean'
    }
)
df_timeline.info()
df_timeline.head(-8)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23652 entries, 0 to 23651
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   LastUpdate         23652 non-null  object 
 1   State              23652 non-null  object 
 2   Confirmed          23652 non-null  float64
 3   Deaths             23652 non-null  float64
 4   IncidenceRate      19360 non-null  float64
 5   CaseFatalityRatio  19360 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.3+ MB


,LastUpdate,State,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio
0,2020-01-22,WA,1.0,0.0,NaN,NaN
1,2020-01-23,WA,1.0,0.0,NaN,NaN
2,2020-01-24,WA,1.0,0.0,NaN,NaN
3,2020-01-25,IL,1.0,0.0,NaN,NaN
4,2020-01-25,WA,1.0,0.0,NaN,NaN
...,...,...,...,...,...,...
23639,2021-05-16,RI,150385.0,2700.0,8878.731293,1.533128
23640,2021-05-16,SC,588110.0,9638.0,11003.661846,2.025390
23641,2021-05-16,SD,123644.0,1991.0,13181.011650,2.179649
23642,2021-05-16,TN,857055.0,12312.0,12520.886518,1.743299


### Total cases per State

In [ ]:
df_total = pd.DataFrame()

df_total = df.groupby('State', as_index=False).agg(
#df_total = df.groupby(['State', 'StateName'], as_index=False).agg(
    {
         'Confirmed': 'max',
         'Deaths': 'max',
         'IncidenceRate': 'mean',
         'CaseFatalityRatio': 'mean'
    }
)

df_total.info()
df_total.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              56 non-null     object 
 1   Confirmed          56 non-null     float64
 2   Deaths             56 non-null     float64
 3   IncidenceRate      55 non-null     float64
 4   CaseFatalityRatio  55 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.6+ KB


,State,Confirmed,Deaths,IncidenceRate,CaseFatalityRatio
0,AK,30555.0,176.0,3026.029499,0.606605
1,AL,79827.0,1529.0,5642.304949,2.134702
2,AR,39378.0,660.0,5706.386278,1.813630
3,AS,0.0,0.0,NaN,NaN
4,AZ,544255.0,9950.0,5707.949725,2.860034


## Load

In [ ]:
# Lo guardo en mi Drive, no puedo guardarlo en el shared drive porque Fer es la owner
# df_us.to_csv('/content/drive/My Drive/010 - Séptimo semestre/Business Intelligence/CovidCases.csv', index = False)

In [ ]:
!ls "/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources"

 CovidCases.csv   CovidTimeline.csv  'Libros de trabajo'   ZCTAs.csv   ZIPs.csv


In [ ]:
df_total.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/CovidCases.csv', index = False)
df_timeline.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/CovidTimeline.csv', index = False)